In [9]:
using HDF5
using PlotlyJS
using Pandas
using Polynomials
#using Unitful
#using Pandas
using FFTW
using LinearAlgebra
#using DifferentialEquations
#using ForwardDiff
#using Interpolations
#using PhysicalConstants.CODATA2018: c_0
#using Debugger
#using TimerOutputs

In [ ]:
ΔM = Array([-2, -1, 0, 1, 2]);
μ = values(df.M - loc(df)[indx_pmp[1]].M);

In [32]:
BraggSplits=[0;;];
loop_modes=[0;;];


µ=[-2, -1, 0, 1, 2];
Bragg_R=2e8


μsplit_Bragg = BraggSplits #.+ μ0 .+ 1
println(μsplit_Bragg[1])
μsplit_Loop = loop_modes#  .+ μ0 .+ 1
println(μsplit_Loop[1])
#for now just set decay of Braggs and loop reflection to be pretty sharp
σ_Bragg = zeros(length(μ),1) .+ .1
σ_Loop = zeros(length(μ),1) .+ .0001
CoupleBragg = 1im*zeros(length(μ),1)
println("µ")
println(µ)
for ii in collect(1:length(μsplit_Bragg))
    CoupleBragg .= CoupleBragg .+ Bragg_R .* exp.(-(µ .- μsplit_Bragg[ii]).^2/(2*σ_Bragg[ii]^2))
end
CoupleBragg

0
0
µ
[-2, -1, 0, 1, 2]


5×1 Matrix{ComplexF64}:
 2.767793053473554e-79 + 0.0im
 3.857499695927863e-14 + 0.0im
                 2.0e8 + 0.0im
 3.857499695927863e-14 + 0.0im
 2.767793053473554e-79 + 0.0im

In [13]:
# -- Allocate FFT to go faster --
ifft_plan = FFTW.plan_ifft(zeros(length(μ), 1))
fft_plan = FFTW.plan_fft(zeros(length(μ), 1))

FFTW forward plan for 5×1 array of ComplexF64
(dft-direct-5 "n1fv_5_neon")

In [30]:
println("Bragg vector")
println(CoupleBragg)
#CoupleBraggShift=CoupleBragg
println("shift")
CoupleBraggShift=FFTW.fftshift(CoupleBragg)#.*length(μ)
println(CoupleBraggShift)
println("no shift ")
CoupleBraggShift=ifft_plan* (CoupleBragg)
println(CoupleBraggShift)

Bragg vector
ComplexF64[2.767793053473554e-79 + 0.0im; 3.857499695927863e-14 + 0.0im; 2.0e8 + 0.0im; 3.857499695927863e-14 + 0.0im; 2.767793053473554e-79 + 0.0im;;]
shift
ComplexF64[3.857499695927863e-14 + 0.0im; 2.767793053473554e-79 + 0.0im; 2.767793053473554e-79 + 0.0im; 3.857499695927863e-14 + 0.0im; 2.0e8 + 0.0im;;]
no shift 
ComplexF64[4.0e7 + 0.0im; -3.2360679774997905e7 + 2.351141009169893e7im; 1.23606797749979e7 - 3.804226065180614e7im; 1.23606797749979e7 + 3.804226065180614e7im; -3.2360679774997905e7 - 2.351141009169893e7im;;]


In [34]:
fpmp=[123]
Ain = [1im*zeros(length(μ),1) for ii in collect(1:length(fpmp))]
Ein = [1im*zeros(length(μ),1) for ii in collect(1:length(fpmp))]
Ppmp=1e-3;
println(Ppmp)
μpmp=0
φpmp=[0]
ind_pmp=[1]
for ii in collect(1:length(fpmp))
    Ein_ = sqrt(Ppmp[ii])*length(μ)
    Ein[ii][Int(μpmp+ind_pmp[ii])] = Ein_
    Ain[ii]= ifft_plan*(FFTW.fftshift(Ein[ii])) .* exp(-1im.*φpmp[ii])
end
println(Ein)
println(Ain)

0.001
Matrix{ComplexF64}[[0.15811388300841894 + 0.0im; 0.0 + 0.0im; 0.0 + 0.0im; 0.0 + 0.0im; 0.0 + 0.0im;;]]
Matrix{ComplexF64}[[0.03162277660168379 + 0.0im; -0.02558336368008464 + 0.018587401723009222im; 0.00977197537924274 - 0.030075047750377278im; 0.00977197537924274 + 0.030075047750377278im; -0.02558336368008464 - 0.018587401723009222im;;]]
